In [3]:
from lxml import etree
import re
from operator import itemgetter


In [4]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open

In [5]:
def extracttext(text):
    
    sentenses = re.split(r"[\n|\r]", text)
    # Remove leading and trailing spaces from each sentence
    results = []
    for sen in sentenses:
        s = sen.strip()
        if len(s):
            results.append(s)
            num_libros=len(results)
    #print("\n")
    return results[1]


In [100]:
def querysearch():
    
    queryid = []
    querytext = []
    a = 1
    queryall = [queryid ,querytext]
    while a!=47:
        if a < 10:
            b = "0"+str(a)
            if a == 4:
                a+=1
        elif a < 50:
                b = str(a)
                if a == 10 or a == 14 or a == 32 or a == 34 or a ==38 or a == 42:
                    a+=1
                elif a == 19 or a == 29:
                    a+=2

        docquer = etree.parse("./queries-raw-texts/wes2015.q"+b+".naf")
        raizquer=docquer.getroot()
        idquer = raizquer[0][1].get('publicId')
        textquery = raizquer[1].text
        queryid.append(idquer)
        querytext.append(textquery)
        a+=1
    #print(queryid)
    #print(querytext)    
    return queryall
    #print("\n")

In [101]:
#Gensim trae sus propias funciones para el procesamiento de texto
p = PorterStemmer()

<h2>Corpus Streaming</h2>
Normalmente los corpus reciden completamente en la memoria. Supongamos que hay millones de documentos en el corpus. Almacenarlos todos en RAM no será suficiente. En su lugar, supongamos que los documentos se almacenan en un archivo en el disco, un documento por línea. Gensim puede procesar un documento a la vez y actualizar dinamicamente e calculo de IDF y del vocabulario.

In [102]:
#Procesamiento de un texto utilizando las funciones de Gensim
def process(text):
    doc_nor = text.lower()
    #doc_reg = re.sub(r'[^\w|^\d]', ' ', doc_nor)
    #doc_reg = re.sub(r'[^a-zA-Z]', ' ', doc_nor)
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = PorterStemmer().stem_sentence(doc_sw)
    return doc_stem.split()

In [123]:
#Creaccion de lista con documentos procesados
docDict = []
docDict1 = []
docDict2 = []
docTitle = []
docID = []
a = 1

while a!=332:
    if a < 10:
        b = "00"+str(a)
    elif a < 100:
            b = "0"+str(a)
    else:
            b = str(a)
    doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
    raiz=doc.getroot()
    titulo = raiz[0][0].get('title')
    docTitle.append(titulo)
    iddoc = raiz[0][1].get('publicId')
    docID.append(iddoc)
    #print(iddoc)
    #print(titulo)
    text = raiz[1].text
    a+=1
    line = extracttext(text)
    #print("\n")
    docDict.append(process(line))

#print(docID[2])
#print(docTitle[2])
#print(docDict[2]) #Documentos ya procesados, de esta forma deben ser enviados al metodo de Gensim



In [124]:
#Ahora si llamamos al metodo de Gensim para crear el diccionario a partir de los documentos
dictionary = corpora.Dictionary(docDict)
dictionary.save('clase3.dict')
#print(dictionary)

#Notese que Gensim de una vez asigna a cada token un id
#print(dictionary.token2id)

In [125]:
new_doc = "Human media interaction media"
new_doc_bow = dictionary.doc2bow(process(new_doc))
print(new_doc_bow)  # ¿Si hay cuatro palabras pq solo dos tokens?

[(70, 1), (4236, 1), (7809, 2)]


In [126]:
#Construccion del corpus == Cada documento representado con el vocabulario/diccionario definido anteriormente (Bolsa de palabras)
class MyCorpus(object):
    def __iter__(self):
        a = 1

        while a!=332:
            if a < 10:
                b = "00"+str(a)
            elif a < 100:
                    b = "0"+str(a)
            else:
                b = str(a)
            doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
            raiz=doc.getroot()
            titulo = raiz[0][0].get('title')
            iddoc = raiz[0][1].get('publicId')
            #print(iddoc)
            #print(titulo)
            text = raiz[1].text
            a+=1
            line = extracttext(text)
            yield dictionary.doc2bow(process(line))

corpus_memory_friendly = MyCorpus()#Todos mis documentos ahora estan representados como una bolsa de palabras
#Almaceno mi corpus
corpora.MmCorpus.serialize('corpus.mm', corpus_memory_friendly)  

In [127]:
#Cargo mi corpus
corpus = corpora.MmCorpus('corpus.mm')
#print(corpus) #No lo carga en memoria
#Para leer la representacion de bolda de palabras resultante de cada documento en el corpus
#for doc in corpus:
#    print(doc)

<h2>Vector Space Model - TF-IDF</h2>

In [128]:
#Suponga que tenemos el diccionario y corpus en disco
#Primero necesitamos cargarlos
dictionary = corpora.Dictionary.load('clase3.dict')
corpus = corpora.MmCorpus('corpus.mm')

In [129]:
#Construyendo un modelo VSM con ponderacion por TF-IDF
tfidf = models.TfidfModel(corpus) #Inizializacion

In [130]:
#Supongamos que queremos calcular la similitud de un nuevo documento con los documentos en el corpus
#1. Construimos la representación vectorial del query
query_doc = "Amazon vs Microsoft"
query_doc_bow = dictionary.doc2bow(process(query_doc))
print(query_doc_bow)
print(tfidf[query_doc_bow])

[(16449, 1)]
[(16449, 1.0)]


In [131]:
#2. Ahora construimos la matrix t/d con los documentos contra los cuales queremos compara el query
index = similarities.MatrixSimilarity(tfidf[corpus]) 
# Vamos a salvar el inidice resultante para no tener que recalcularlo cada vez que los necesitemos.
index.save('clase3tfidf.index')

In [132]:
#Una vez guardado lo podemos cargar sin necesidad de recalcularlo
index = similarities.MatrixSimilarity.load('clase3tfidf.index')

In [133]:
# Ejecutar las similitudes entre el documento query y el corpus ahora es muy facil!!!

queryreturn = querysearch()
queryid = queryreturn[0]
querydoc = queryreturn[1]
archivo = open ('top-10.txt','w')
id=0
for query in querydoc:
    
    query_doc_bow = dictionary.doc2bow(process(query))
    sims = index[tfidf[query_doc_bow]]
    #print(type(sims))
    listofdoctf= list(enumerate(sims))
    #print(listofdoctf)
    #print("\n")
    sorted_by_second = sorted(listofdoctf,key=itemgetter(1), reverse = True)
    #print(sorted_by_second)
    varTen = "10 most similarity documents:\n"
    #archivo.write(varTen)
    archivo.write(queryid[id]+"\t")
    for doc, similitudQ in sorted_by_second[:10]:  
        archivo.write( docID[doc]+",")
        #print("{}\t{}".format(docID[doc], similitudQ))
    #print("\n")    
    archivo.write("\n")
    id+=1
archivo.close()
    

#print(list(enumerate(sims)))  print (document_number, document_similarity) 2-tuples#

In [204]:
for line in smart_open('relevance-judgments.tsv', 'rb'):
    docDict2.append(process(line))  
#print(docDict2)

In [205]:
data22 = docDict2[0][1]
data22

'd186:4,d254:5,d016:5'

In [208]:
sentence = re.sub(r':\d', '', data22)
sentence = re.split(",",sentence)
sentence

['d186', 'd254', 'd016']

In [210]:
sentence[1]

'd254'

In [181]:
for line in smart_open('top-10.txt', 'rb'):
    docDict1.append(process(line))  
print(docDict1)

[['q01', 'd016,d259,d209,d254,d085,d186,d170,d008,d315,d004,'], ['q02', 'd147,d283,d002,d134,d293,d023,d014,d282,d143,d316,'], ['q03', 'd283,d152,d001,d002,d003,d004,d005,d006,d007,d008,'], ['q04', 'd270,d275,d310,d019,d010,d049,d100,d060,d205,d111,'], ['q06', 'd297,d329,d026,d029,d257,d025,d233,d166,d154,d023,'], ['q07', 'd289,d146,d004,d034,d266,d123,d098,d284,d042,d234,'], ['q08', 'd110,d251,d292,d106,d107,d180,d108,d246,d117,d235,'], ['q09', 'd217,d198,d223,d085,d177,d330,d299,d069,d175,d065,'], ['q10', 'd060,d231,d100,d036,d052,d031,d072,d091,d034,d314,'], ['q12', 'd277,d258,d239,d250,d056,d132,d081,d078,d012,d096,'], ['q13', 'd272,d049,d276,d056,d258,d239,d277,d043,d286,d241,'], ['q14', 'd145,d002,d091,d121,d081,d122,d030,d180,d117,d133,'], ['q16', 'd132,d184,d250,d176,d277,d239,d081,d024,d078,d156,'], ['q17', 'd271,d172,d146,d091,d121,d024,d171,d274,d183,d316,'], ['q18', 'd192,d194,d201,d230,d207,d210,d111,d216,d223,d222,'], ['q19', 'd179,d323,d004,d273,d021,d102,d001,d330,d069,

In [192]:
data12 = docDict1[0][1]
data12


'd016,d259,d209,d254,d085,d186,d170,d008,d315,d004,'

In [200]:
sentence1 = re.split(",",data12)
sentence1

['d016',
 'd259',
 'd209',
 'd254',
 'd085',
 'd186',
 'd170',
 'd008',
 'd315',
 'd004',
 '']

In [202]:
sentence1[1]

'd259'